In [36]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

from tqdm import tqdm
import csv
import os
import requests
import time

import pandas as pd

In [37]:
FIRST_RUN = False
cotation_url = ['https://www.slickcharts.com/sp500']

In [38]:
def scrape_isin(urls):
    driver = webdriver.Chrome()
    for url in urls:
        driver.get(url)
        # cookies = driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
        # wait for loading and then click on the button
        cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')))
        cookies.click()
        
        try:
            # get the table
            table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div[2]/div[1]/div/div/table')))
            # get the rows
            rows = table.find_elements(By.TAG_NAME, 'tr')
            symbols = [row.find_elements(By.TAG_NAME, 'td')[2].text for row in rows[1:]]
            # get the headers
            headers = rows[0].find_elements(By.TAG_NAME, 'th')
        
            # get the isin
        except NoSuchElementException:
            print("Table element not found.")
        
        #Save the list in a txt file
        with open('../data/symbols.txt', 'w') as f:
            for item in symbols:
                f.write(item + '\n')
                
        return symbols

In [39]:
if FIRST_RUN:
    symbols = scrape_isin(cotation_url)
else:
    with open('../data/symbols.txt', 'r') as f:
        symbols = f.readlines()
        symbols = [symbol.strip() for symbol in symbols]

In [40]:
symbols

['MSFT',
 'AAPL',
 'NVDA',
 'AMZN',
 'META',
 'GOOGL',
 'GOOG',
 'BRK.B',
 'LLY',
 'AVGO',
 'JPM',
 'XOM',
 'UNH',
 'V',
 'TSLA',
 'MA',
 'PG',
 'JNJ',
 'HD',
 'MRK',
 'COST',
 'ABBV',
 'CVX',
 'CRM',
 'NFLX',
 'WMT',
 'AMD',
 'BAC',
 'PEP',
 'KO',
 'LIN',
 'ADBE',
 'TMO',
 'DIS',
 'WFC',
 'ACN',
 'MCD',
 'CSCO',
 'ORCL',
 'ABT',
 'QCOM',
 'CAT',
 'INTU',
 'GE',
 'IBM',
 'VZ',
 'AMAT',
 'DHR',
 'CMCSA',
 'COP',
 'NOW',
 'INTC',
 'TXN',
 'UBER',
 'PFE',
 'AMGN',
 'PM',
 'UNP',
 'RTX',
 'SPGI',
 'LOW',
 'GS',
 'ISRG',
 'NEE',
 'MU',
 'HON',
 'AXP',
 'ETN',
 'PGR',
 'BKNG',
 'LRCX',
 'ELV',
 'NKE',
 'T',
 'SYK',
 'MS',
 'C',
 'SCHW',
 'TJX',
 'BLK',
 'MDT',
 'DE',
 'UPS',
 'VRTX',
 'CI',
 'ADP',
 'BSX',
 'CB',
 'PLD',
 'LMT',
 'BA',
 'SBUX',
 'MMC',
 'BMY',
 'REGN',
 'ADI',
 'MDLZ',
 'KLAC',
 'BX',
 'FI',
 'PANW',
 'CVS',
 'TMUS',
 'GILD',
 'SNPS',
 'AMT',
 'CDNS',
 'CMG',
 'EOG',
 'SO',
 'TGT',
 'WM',
 'CME',
 'ICE',
 'MPC',
 'SHW',
 'DUK',
 'MO',
 'SLB',
 'FCX',
 'CL',
 'ZTS',
 'PH',
 '

In [63]:
def scrape_financial(symbols, progress_file):
    links = []
    driver = webdriver.Chrome()
    with open(progress_file, 'r') as f:
        progress = f.readlines()
        progress = [line.split(':')[0] for line in progress]

    symbols = [symbol for symbol in symbols if symbol not in progress]
    for symbol in symbols:
        url = f'https://fr.finance.yahoo.com/quote/{symbol}/history'
        driver.get(url)
        driver.delete_all_cookies()
        
        try:
            try:
                cookies = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[2]')))
                cookies.click()
            except :
                pass
            
            date = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div')
            date.click()
            
            date_select = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[1]/div[1]/div[1]/div/div/div[2]/div/ul[2]/li[4]/button/span')
            date_select.click()
            
            confirm = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[1]/div[1]/button')
            confirm.click()
            
            try : 
                link = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a')
                download_link = link.get_attribute('href')
                
                links.append(download_link)
                with open (progress_file, 'a') as f:
                    f.write(symbol + ':' + download_link + '\n')
            except :
                pass
        except:
            #Quit driver
            driver.quit()
            scrape_financial(symbols, progress_file='../data/links.txt')
        
    return links
    
links = scrape_financial(symbols, progress_file='../data/links.txt')

In [64]:
len(symbols)

503